# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data=data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})

In [5]:
data=data.reset_index()

In [6]:
data.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [7]:
#data=data.T
data_step2=data.pivot(index ='ProductName', columns ='CustomerID', values ='Quantity') 

In [8]:
data_step2.fillna(0, inplace = True) 

In [9]:
data_step2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [10]:
from scipy.spatial.distance import pdist, squareform
#similarity_matrix = pdist(data, 'euclidean')

In [11]:
similarity_m = pd.DataFrame(1/(1 + squareform(pdist(data_step2.T, 'euclidean'))), 
                         index=data_step2.columns, columns=data_step2.columns)

In [12]:
similarity_m

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
464,0.082709,0.075255,0.080350,0.078187,0.078711,1.000000,0.074798,0.084626,0.080070,0.078187,...,0.004816,0.004687,0.004416,0.005038,0.004308,0.004533,0.004578,0.004363,0.004167,0.004347
477,0.074573,0.075956,0.076923,0.075025,0.075025,0.074798,1.000000,0.078977,0.075720,0.074573,...,0.004823,0.004688,0.004426,0.005037,0.004321,0.004534,0.004589,0.004366,0.004168,0.004338
639,0.083020,0.076435,0.080634,0.082403,0.082403,0.084626,0.078977,1.000000,0.085983,0.083020,...,0.004816,0.004682,0.004412,0.005035,0.004308,0.004526,0.004581,0.004363,0.004176,0.004335
649,0.081503,0.077674,0.082100,0.077171,0.078187,0.080070,0.075720,0.085983,1.000000,0.079792,...,0.004810,0.004676,0.004414,0.005035,0.004314,0.004528,0.004585,0.004369,0.004172,0.004329


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [13]:
#clientes similares al CustomerID 33

similarities = similarity_m[33].sort_values(ascending=False)[1:5]
similarities

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
Name: 33, dtype: float64

In [14]:
top5=[s for s in similarities.index]
top5

[264, 3535, 3317, 2503]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [15]:
data.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [16]:
#filtrar por los customers de arriba CustomerID
filtered=data[data['CustomerID'].isin(top5)]
filtered.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [17]:
step6=filtered.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending=False)
step6.head()

,Quantity
ProductName,
"Pepper - Black, Whole",3
Soup - Campbells Bean Medley,3
Butter - Unsalted,3
Wine - Blue Nun Qualitatswein,2
"Nut - Chestnuts, Whole",2


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [18]:
#crear otro dataframe con como el anterior pero solo del customer 33 y hacer un concat.
filtered_33 = data[(data['CustomerID']==33)]
filtered_33

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
5,33,"Beef - Chuck, Boneless",1
6,33,Beef - Prime Rib Aaa,1
7,33,Beer - Original Organic Lager,1
8,33,Beer - Rickards Red,1
9,33,Black Currants,1


In [19]:
filtered_33=filtered_33.groupby(['ProductName']).agg({'Quantity':'sum'})

In [20]:
filtered_33.rename(columns={'Quantity':'Quantity_cust33'},inplace=True)
filtered_33.head()

,Quantity_cust33
ProductName,
Apricots - Dried,1
Assorted Desserts,1
Bandage - Flexible Neon,1
"Bar Mix - Pina Colada, 355 Ml",1
"Beans - Kidney, Canned",1


In [21]:
mergead=pd.concat([step6, filtered_33])

/home/eduardo/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [22]:
mergead = mergead[mergead.iloc[:,1].isnull()]
mergead.head()

,Quantity,Quantity_cust33
ProductName,,
"Pepper - Black, Whole",3.0,NaN
Soup - Campbells Bean Medley,3.0,NaN
Butter - Unsalted,3.0,NaN
Wine - Blue Nun Qualitatswein,2.0,NaN
"Nut - Chestnuts, Whole",2.0,NaN


In [23]:
mergead.iloc[0:5,0].index

Index(['Pepper - Black, Whole', 'Soup - Campbells Bean Medley',
       'Butter - Unsalted', 'Wine - Blue Nun Qualitatswein',
       'Nut - Chestnuts, Whole'],
      dtype='object', name='ProductName')

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [24]:
recomendations={}
CustomerIDs=list(set([cust for cust in data['CustomerID']]))
for customer in CustomerIDs:
    similarities = similarity_m[customer].sort_values(ascending=False)[1:5]
    top_five=[s for s in similarities.index]
    filtered=data[data['CustomerID'].isin(top_five)]
    step6=filtered.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending=False)
    filter_customer = data[(data['CustomerID']==customer)]
    filter_customer2=filter_customer.groupby(['ProductName']).agg({'Quantity':'sum'})
    mergeado=pd.concat([step6, filter_customer2],axis=1,sort=False)
    mergeado = mergeado[mergeado.iloc[:,1].isnull()]
    recomendation=mergeado.iloc[0:5,0].index
    recomendations[customer] = recomendation



In [25]:
recomendations

{83973: Index(['Cocoa Butter', 'Veal - Slab Bacon', 'Bandage - Fexible 1x3',
        'Ezy Change Mophandle', 'Milk Powder'],
       dtype='object'),
 59399: Index(['Shrimp - Baby, Warm Water', 'Nantucket - Pomegranate Pear',
        'Bagel - Plain', 'Towels - Paper / Kraft', 'Snapple - Iced Tea Peach'],
       dtype='object'),
 92168: Index(['Cup - Translucent 7 Oz Clear', 'Veal - Brisket, Provimi,bnls',
        'Snapple Lemon Tea', 'Bread - Calabrese Baguette', 'Spinach - Baby'],
       dtype='object'),
 18441: Index(['Soupfoamcont12oz 112con', 'Sausage - Breakfast',
        'Beef - Texas Style Burger', 'Flour - Pastry', 'Foam Cup 6 Oz'],
       dtype='object'),
 60759: Index(['Cheese - Wine', 'Beef Wellington', 'Nut - Chestnuts, Whole',
        'Appetizer - Mini Egg Roll, Shrimp', 'Wine - White Cab Sauv.on'],
       dtype='object'),
 86028: Index(['Spinach - Baby', 'Pork - Hock And Feet Attached', 'Vinegar - Tarragon',
        'Pepsi - Diet, 355 Ml', 'Truffle Cups - Brown'],
       d

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [26]:
df_recomendations=pd.DataFrame(recomendations)

In [27]:
df_recomendations.T

,0,1,2,3,4
33,Soup - Campbells Bean Medley,Butter - Unsalted,Wine - Blue Nun Qualitatswein,"Nut - Chestnuts, Whole",Chocolate - Dark
200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Pork - Kidney,Soupfoamcont12oz 112con,Foam Dinner Plate
264,Bread - Italian Roll With Herbs,Wine - Two Oceans Cabernet,"Veal - Inside, Choice",Soupfoamcont12oz 112con,Water - Green Tea Refresher
356,"Veal - Inside, Choice",Butter - Unsalted,Wasabi Powder,Cheese - Bocconcini,Lamb - Ground
412,"Pepper - Black, Whole",Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Corn Meal,Scallops 60/80 Iqf
464,Butter - Unsalted,Bar - Granola Trail Mix Fruit Nut,Sausage - Breakfast,"Pepper - Black, Whole",Yoghurt Tubes
477,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,Corn Meal,Scallops 60/80 Iqf,"Tart Shells - Sweet, 4"
639,Cod - Black Whole Fillet,Wine - Ej Gallo Sierra Valley,"Wine - Magnotta, Merlot Sr Vqa","Pepper - Black, Whole",Towels - Paper / Kraft
649,Appetizer - Mushroom Tart,Bar - Granola Trail Mix Fruit Nut,"Fish - Scallops, Cold Smoked",Flavouring - Orange,Extract - Lemon
669,"Veal - Inside, Choice","Pepper - Black, Whole",Anchovy Paste - 56 G Tube,"Tart Shells - Sweet, 4",Cheese Cloth No 100


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [30]:
similarity_m2 = pd.DataFrame(1/(1 + squareform(pdist(data_step2.T, 'cosine'))), 
                         index=data_step2.columns, columns=data_step2.columns)

In [31]:
similarity_m2

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636
464,0.541054,0.519193,0.525368,0.523986,0.534073,1.000000,0.528047,0.546574,0.528571,0.527240,...,0.537619,0.544446,0.534295,0.535278,0.526394,0.542873,0.520835,0.533856,0.528842,0.552221
477,0.522016,0.556945,0.538967,0.536792,0.542308,0.528047,1.000000,0.547544,0.534712,0.536302,...,0.543155,0.541855,0.545908,0.530540,0.544731,0.540381,0.534221,0.535601,0.527943,0.532238
639,0.530783,0.516729,0.514932,0.540890,0.547743,0.546574,0.547544,1.000000,0.555194,0.548386,...,0.539687,0.538500,0.528522,0.532803,0.527813,0.533190,0.525789,0.535704,0.548943,0.531706
649,0.533096,0.536350,0.537046,0.516901,0.530459,0.528571,0.534712,0.555194,1.000000,0.538236,...,0.529649,0.528778,0.530655,0.531114,0.537025,0.535224,0.531918,0.544926,0.539210,0.519657


In [32]:
recomendations2={}
CustomerIDs=list(set([cust for cust in data['CustomerID']]))
for customer in CustomerIDs:
    similarities = similarity_m2[customer].sort_values(ascending=False)[1:5]
    top_five=[s for s in similarities.index]
    filtered=data[data['CustomerID'].isin(top_five)]
    step6=filtered.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending=False)
    filter_customer = data[(data['CustomerID']==customer)]
    filter_customer2=filter_customer.groupby(['ProductName']).agg({'Quantity':'sum'})
    mergeado=pd.concat([step6, filter_customer2],axis=1,sort=False)
    mergeado = mergeado[mergeado.iloc[:,1].isnull()]
    recomendation=mergeado.iloc[0:5,0].index
    recomendations2[customer] = recomendation

In [33]:
recomendations2

{83973: Index(['Cheese - Brie,danish', 'Pastry - Choclate Baked',
        'Puree - Passion Fruit', 'Lettuce - Spring Mix', 'Broom - Corn'],
       dtype='object'),
 59399: Index(['Anchovy Paste - 56 G Tube', 'Ezy Change Mophandle', 'Garlic - Peeled',
        'Broom - Corn', 'Cake - Mini Cheesecake'],
       dtype='object'),
 92168: Index(['Beer - Alexander Kieths, Pale Ale', 'Nantucket - Pomegranate Pear',
        'Tia Maria', 'Wine - Red, Colio Cabernet', 'Tuna - Salad Premix'],
       dtype='object'),
 18441: Index(['V8 - Berry Blend', 'Whmis - Spray Bottle Trigger',
        'Tea - Jasmin Green', 'Beef - Inside Round',
        'Pasta - Penne, Rigate, Dry'],
       dtype='object'),
 60759: Index(['Sobe - Tropical Energy', 'Table Cloth 81x81 White',
        'Pastry - Choclate Baked', 'Ice Cream Bar - Drumstick',
        'Soup - Campbells, Lentil'],
       dtype='object'),
 86028: Index(['Mayonnaise - Individual Pkg', 'Pail With Metal Handle 16l White',
        'Sunflower Seed Raw', 'Or

In [34]:
df_recomendations2=pd.DataFrame(recomendations2)

In [35]:
df_recomendations2.T

,0,1,2,3,4
33,Knife Plastic - White,Banana Turning,Muffin - Zero Transfat,Chips Potato Salt Vinegar 43g,"Thyme - Lemon, Fresh"
200,General Purpose Trigger,Sponge Cake Mix - Chocolate,Juice - Lime,Langers - Ruby Red Grapfruit,Muffin Batt - Blueberry Passion
264,Pickerel - Fillets,"Water - Mineral, Natural",Snapple - Iced Tea Peach,Wine - Ej Gallo Sierra Valley,French Pastry - Mini Chocolate
356,Cheese - Taleggio D.o.p.,"Coconut - Shredded, Sweet",Cheese - Cheddarsliced,Ocean Spray - Kiwi Strawberry,Olives - Kalamata
412,"Salsify, Organic",Durian Fruit,Bread - Raisin Walnut Oval,Gatorade - Xfactor Berry,Sea Bass - Whole
464,Wonton Wrappers,Bread - Bistro White,Sausage - Breakfast,Soup Campbells - Italian Wedding,Garbage Bags - Clear
477,"Pasta - Penne, Rigate, Dry",Olive - Spread Tapenade,Veal - Osso Bucco,Mussels - Cultivated,"Cheese - Boursin, Garlic / Herbs"
639,Crab - Imitation Flakes,Apricots Fresh,"Pepsi - Diet, 355 Ml",Crush - Cream Soda,Garbage Bags - Clear
649,"Mushroom - Porcini, Dry",Bay Leaf,"Fish - Scallops, Cold Smoked",Browning Caramel Glace,Napkin White - Starched
669,Yoghurt Tubes,Chicken - Wieners,Mussels - Frozen,Cheese - Cheddarsliced,Juice - V8 Splash
